In [57]:
import cv2
import numpy as np
import os
import time
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from pymodbus.client import ModbusTcpClient

In [58]:
cam = 2

model_version = "my_model_23.1.keras"

Im_Width = 128
Im_Height = 128

Y6 = 1542 #ModBus Y6
Y4 = 1540
PLC_IP = "192.168.1.111"  #PLC's IP address
PLC_PORT = 502           # Default Modbus TCP port

In [59]:
def modbus(PLC_IP, PLC_PORT):

    # Create Modbus client
    client = ModbusTcpClient(PLC_IP, port=PLC_PORT)

    # Connect to the PLC
    if client.connect():
        print("Connection successful")
    else:
        print("Failed to connect to PLC")
    return client

In [60]:
def read_PLC_output(client, coil):
    read_result = client.read_coils(address=coil)  # Read a single coil (output)
    if read_result.isError():
        print("Read Error:", read_result)
        return None
    return read_result.bits[0]  # Return the first coil state (True for ON, False for OFF)

In [61]:
def write_PLC_output(client, coil,value):
    write_result = client.write_coil(address=coil, value=value)  # Write 'True' to turn ON
    if write_result.isError():
        print("Write Error:", write_result)
    else:
        print(f"Output Y6 turned ON (Coil {Y6})")

In [62]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [63]:
def predict_frame(client, frame, model, target_size):
    
    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    predictions = model.predict(frame_array)  # Outputs probabilities for each class
    predicted_class = np.argmax(predictions[0])  # Get the index of the class with the highest probability
    confidence = predictions[0][predicted_class]  # Confidence score for the predicted class

    # Map class indices to labels
    class_labels = {0: "Defective", 2: "Non-Defective", 1: "No Bottle"}  # Update this to match your classes
    predicted_label = class_labels[predicted_class]

    #prob = abs(round(confidence,2)- 0.5)*200

    if predicted_class == 0 and confidence > 50:
        write_PLC_output(client, Y6, False)
        
    # elif predicted_class == 1:
    #     write_PLC_output(client, Y6, True)
    #     time.sleep(0.1)
    #     write_PLC_output(client, Y6, False)

    # elif predicted_class == 2:
    #     write_PLC_output(client, Y6, True)
    #     time.sleep(0.1)
    #     write_PLC_output(client, Y6, False)

    else:
        write_PLC_output(client, Y6, True)
        time.sleep(0.2)
        write_PLC_output(client, Y6, False)

    print(f"Raw Predictions: {predictions[0]}")
    print(f"Predicted Class: {predicted_label}, Confidence: {confidence:.2f}")

    return predicted_label, confidence

In [64]:
def capture(client, model_version, target_size=(Im_Width, Im_Height)):
    cap = cv2.VideoCapture(2)
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

     # Load the model
    print("Loading model...") 
    model = load_model(model_version)
    print("Model loaded successfully.")
    
    output_dir = "frames"
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    frame_count = 0
    #flag = False  # Tracks the previous state of the output

    while True:
        result_Y4 = read_PLC_output(client, Y4)  # Ensure count=1 to avoid errors
        #result_Y4 = read_PLC_output(client, Y6)
        # if result.isError():
        #     print("Modbus Read Error:", result)
        #     continue  # Skip iteration on error

        #current_state = result  # Read the coil status

        # Detect rising edge: transition from False -> True
        if result_Y4:
            write_PLC_output(client, Y6, False)
            print("TRUE - Capturing Frame")
            time.sleep(1)
            ret, frame = cap.read()
            if not ret:
                print("Error: Frame capture failed.")
                break
            
            # Save the frame
            frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved: {frame_filename}") 
            frame_count += 1

            predicted_label, confidence = predict_frame(client, frame, model, target_size=(Im_Width, Im_Height))

            prob = abs(round(confidence,2)- 0.5)*200
            print(f"Prediction: {predicted_label} (Confidence: {confidence:.2f})")

            # Display predicted_label on the frame
            cv2.putText(frame, f"Prediction: {predicted_label}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {prob:.0f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Display the best frame
            cv2.imshow("Best Frame", frame)

        # Update flag to track previous state
        #flag = current_state

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


In [65]:
if __name__ == "__main__":

    client = modbus(PLC_IP, PLC_PORT)
    capture(client, model_version, target_size=(Im_Width, Im_Height))


Connection successful
Loading model...
Model loaded successfully.
Output Y6 turned ON (Coil 1542)
TRUE - Capturing Frame
Saved: frames\frame_0000.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step
Output Y6 turned ON (Coil 1542)
Output Y6 turned ON (Coil 1542)
Raw Predictions: [6.1491364e-01 4.2831807e-06 3.8508213e-01]
Predicted Class: Defective, Confidence: 0.61
Prediction: Defective (Confidence: 0.61)
Output Y6 turned ON (Coil 1542)
TRUE - Capturing Frame
Saved: frames\frame_0001.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Output Y6 turned ON (Coil 1542)
Raw Predictions: [6.9074893e-01 2.4498165e-06 3.0924863e-01]
Predicted Class: Defective, Confidence: 0.69
Prediction: Defective (Confidence: 0.69)
Output Y6 turned ON (Coil 1542)
TRUE - Capturing Frame
Saved: frames\frame_0002.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Output Y6 turned ON (Coil 1542)
Raw Predictions: [6.9638377e-01 1.8730144e-06 3.0361435e-01]
Predicted Class: Defective, Confidence: 0.70
Prediction: Defective (Confidence: 0.70